<a href="https://colab.research.google.com/github/nanopiero/fusion/blob/main/notebooks/fcns/training_B11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## B1.1 radar + gauges 1 min + cmls -> gauges 1 min [xrlg1_yg1]


In [ ]:
! git clone https://github.com/nanopiero/fusion.git

In [8]:
! pwd

/home/mdso/lepetitp/ppc/WEBCAMS/src/raincell/ia/notebooks/learning/simulation/fusion/notebooks/fcns


In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import os
import time
import sys
sys.path.append('/home/mdso/lepetitp/ppc/WEBCAMS/src/raincell/ia/notebooks/learning/simulation')

from fusion.utils.datasets import spatialized_gt, create_cmls_filter, FusionDataset
from fusion.utils.datasets import indices_to_sampled_values, get_point_measurements, point_gt, segment_gt, make_noisy_images
from torch.utils.data import DataLoader
from fusion.utils.fcn import UNet
from fusion.utils.cost_functions import QPELoss_fcn, compute_metrics
from fusion.utils.viz import set_tensor_values2, plot_images, plot_images_10pts_20seg, plot_results_10pts_20seg

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Config
num_epochs = 2000
save_every = 10
path = r'/scratch/mdso/lepetitp/ppc/RAINCELL/models/simulation/checkpoint_fcn_exp_B11_xrlg1_yg1.pt'
npairs = 20
nsteps = 60
ndiscs = 5
size_image=64
length_dataset = 6400
device = torch.device('cuda:0')

# Entraînement
npoints = 20
dataset = FusionDataset(length_dataset=length_dataset,
                        npairs=npairs,
                        nsteps=nsteps,
                        ndiscs=ndiscs, size_image=size_image)


loader = DataLoader(dataset, batch_size=64, num_workers=4)


In [4]:
# Tiny UNet V1. 60 new channels for input time series of rain gauges measurements
use_fcn = True
ch_in = 72 + 60
ch_out = nsteps * 3 + 1
size = nsteps * 3

model = UNet(ch_in, ch_out, size, nb_additional_parameters=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = QPELoss_fcn()

In [5]:
# Boucle avec 5 modes d'évaluation
val_steps = ['eval_opportunity_cost_spat',
             # 'eval_added_value_few_spat',
             # 'eval_added_value_half_spat',
             'eval_added_value_full_spat',
             # 'eval_added_value_full_id',
             ]
steps = val_steps + ['train']
losses = {step:[] for step in steps}
last_epoch = 0

In [7]:
checkpoint = torch.load(path, \
                            map_location=device)
last_epoch = checkpoint['epoch']
losses = checkpoint['train_losses']
# best_loss = checkpoint['best_loss']
model_weights = checkpoint['model']
optimizer_state_dict = checkpoint['optimizer']
# scheduler_state_dict = checkpoint['scheduler']
model.load_state_dict(model_weights)
optimizer.load_state_dict(optimizer_state_dict)
# scheduler.load_state_dict(scheduler_state_dict)

In [8]:
last_epoch

1480

In [ ]:
model.train()

for epoch in range(last_epoch, num_epochs + 1):
  t = time.time()
  print('epoch n°', epoch, '\n')

  running_regression_loss = {step:0.0 for step in steps}
  running_regression_loss_1h = {step:0.0 for step in steps}
  running_segmentation_loss = {step:0.0 for step in steps}
  running_confusion_matrix = {step: np.zeros((2, 2), dtype=int) for step in steps}

  for i, (images, pairs, filters) in enumerate(loader):

    # ground truth (not usable)
    images = images.clone().detach().float().to(device)

    # pseudo radar
    noisy_images = make_noisy_images(images)

    # pseudo CMLs
    pairs = pairs.clone().detach().float().to(device)
    filters = filters.clone().float().detach().to(device)

    # segment_measurements = segment_gt(images, pairs, filters)
    _, segment_measurements_fcn = segment_gt(images, pairs, filters,
                                             use_fcn=use_fcn)

    #Validation steps
    model.eval()
    with torch.no_grad():

      # splitting
      np_val_inputs_few = 1
      np_val_inputs_half = npoints//4  # 5
      np_val_inputs_comp = npoints//2 - np_val_inputs_half - np_val_inputs_few  # 4
      np_val_targets = npoints//2 # 10

      # val split
      split_val = [np_val_inputs_few, np_val_inputs_half, np_val_inputs_comp, np_val_targets]
      split_few, split_half, split_comp, split_targets = point_gt(images, npoints=npoints,
                                                                    use_fcn=use_fcn,
                                                                    split=split_val)
      _, point_measurements_fcn_eval_few, _ = split_few
      _, point_measurements_fcn_eval_half, _ = split_half
      _, point_measurements_fcn_eval_comp, _ = split_comp
      _, point_measurements_fcn_val_targets, _ = split_targets

      # 4 first val steps (10 last pluvios for testing generalization)
      targets = point_measurements_fcn_val_targets

      # val step 1 : eval_opportunity_cost_spat
      step = 'eval_opportunity_cost_spat'
      inputs = torch.cat([noisy_images,
                          segment_measurements_fcn,
                          0 * point_measurements_fcn_eval_few - 0.1
                          ], dim=1)
      outputs = model(inputs)
      regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _ = criterion(model.p, outputs, targets)
      running_regression_loss[step] += regression_loss
      running_regression_loss_1h[step] += regression_loss_1h
      running_segmentation_loss[step] += segmentation_loss
      running_confusion_matrix[step] += batch_cm

      del inputs, outputs, loss, regression_loss, regression_loss_1h, segmentation_loss
      torch.cuda.empty_cache()

      # val step 2 : eval_added_value_few_spat
      # step = 'eval_added_value_few_spat'
      # inputs = torch.cat([noisy_images,
      #                     segment_measurements_fcn,
      #                     point_measurements_fcn_eval_few
      #                     ], dim=1)
      # outputs = model(inputs)
      # regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm = criterion(model.p, outputs, targets)
      # running_regression_loss[step] += regression_loss
      # running_regression_loss_1h[step] += regression_loss_1h
      # running_segmentation_loss[step] += segmentation_loss
      # running_confusion_matrix[step] += batch_cm

      # del inputs, outputs, loss, regression_loss, regression_loss_1h, segmentation_loss
      # torch.cuda.empty_cache()

      # val step 3 : eval_added_value_half_spat
      # step = 'eval_added_value_half_spat'
      # inputs = torch.cat([noisy_images,
      #                     segment_measurements_fcn,
      #                     point_measurements_fcn_eval_half
      #                     ], dim=1)
      # outputs = model(inputs)
      # regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm = criterion(model.p, outputs, targets)
      # running_regression_loss[step] += regression_loss
      # running_regression_loss_1h[step] += regression_loss_1h
      # running_segmentation_loss[step] += segmentation_loss
      # running_confusion_matrix[step] += batch_cm

      # del inputs, outputs, loss, regression_loss, regression_loss_1h, segmentation_loss
      # torch.cuda.empty_cache()

      # val step 4 : eval_added_value_full_spat
      step = 'eval_added_value_full_spat'

      point_measurements_fcn_eval_full = point_measurements_fcn_eval_few + \
                                          point_measurements_fcn_eval_half + \
                                          point_measurements_fcn_eval_comp

      point_measurements_fcn_eval_full += 2 * 0.1 * (point_measurements_fcn_eval_few >= 0)
      point_measurements_fcn_eval_full += 2 * 0.1 * (point_measurements_fcn_eval_half >= 0)
      point_measurements_fcn_eval_full += 2 * 0.1 * (point_measurements_fcn_eval_comp >= 0)

      point_measurements_fcn_eval_full[point_measurements_fcn_eval_full<0] = -0.1

      inputs = torch.cat([noisy_images,
                          segment_measurements_fcn,
                          point_measurements_fcn_eval_full
                          ], dim=1)

      outputs = model(inputs)
      regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _ = criterion(model.p, outputs, targets)
      running_regression_loss[step] += regression_loss
      running_regression_loss_1h[step] += regression_loss_1h
      running_segmentation_loss[step] += segmentation_loss
      running_confusion_matrix[step] += batch_cm

      del inputs, loss, regression_loss, regression_loss_1h, segmentation_loss, split_few, split_half, split_comp, split_targets
      torch.cuda.empty_cache()

      # last val step, on the 10 first pluvios : eval_added_value_full_id
      # step = 'eval_added_value_full_id'
      # regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm = criterion(model.p, outputs, point_measurements_fcn_eval_full) # inputs pluvios serve as targets
      # running_regression_loss[step] += regression_loss
      # running_regression_loss_1h[step] += regression_loss_1h
      # running_segmentation_loss[step] += segmentation_loss
      # running_confusion_matrix[step] += batch_cm

      # del outputs, loss, regression_loss, regression_loss_1h, segmentation_loss
      del point_measurements_fcn_eval_full
      torch.cuda.empty_cache()

    # train step
    model.train()
    step = 'train'
    np_train_inputs = torch.randint(0,9,(1,))
    np_train_targets = npoints // 2 - np_train_inputs
    split_train = [np_train_inputs, np_train_targets]


    # split  train
    split_inputs, split_targets = point_gt(images, npoints=npoints,
                                           use_fcn=use_fcn,
                                           split=split_train)

    _, point_measurements_fcn_train_inputs, _ = split_inputs
    _, point_measurements_fcn_train_targets, _ = split_targets

    inputs = torch.cat([noisy_images,
                        segment_measurements_fcn,
                        point_measurements_fcn_train_inputs
                        ], dim=1)
    targets = point_measurements_fcn_train_targets

    optimizer.zero_grad()  # Zero the gradients
    outputs = model(inputs)
    regression_loss, regression_loss_1h, segmentation_loss, loss, batch_cm, _ = criterion(model.p, outputs, targets)

    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

    running_regression_loss[step] += regression_loss
    running_regression_loss_1h[step] += regression_loss_1h
    running_segmentation_loss[step] += segmentation_loss
    running_confusion_matrix[step] += batch_cm

    del split_inputs, inputs, outputs, split_targets, loss, regression_loss, regression_loss_1h, segmentation_loss, noisy_images, images, pairs, filters, segment_measurements_fcn
    torch.cuda.empty_cache()

  if epoch > 0:
    for step in steps:
      regression_loss = running_regression_loss[step] / len(loader)
      regression_loss_1h = running_regression_loss_1h[step] / len(loader)
      segmentation_loss = running_segmentation_loss[step] / len(loader)
      losses[step].append((epoch, regression_loss, regression_loss_1h, segmentation_loss, running_confusion_matrix[step]))

      print(f'{step}, Regression Loss: {regression_loss:.4f}, Regression Loss 1h: {regression_loss_1h:.4f}, Segmentation Loss:{segmentation_loss:.4f}' )
      print("Train Confusion Matrix:")
      print(running_confusion_matrix[step])
      accuracy, csi, sensitivity, specificity, false_alarm_ratio = compute_metrics(running_confusion_matrix[step])
      print(f'Accuracy: {accuracy:.4f}, CSI: {csi:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}, False Alarm Ratio: {false_alarm_ratio:.4f}')
      print('\n')
  print('epoch duration :', time.time() - t)

  if (epoch % save_every == 0 or \
    epoch == last_epoch):
    print("saving step")
    checkpoint = { 
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        # 'scheduler': scheduler.state_dict(),
        'train_losses': losses,
        }
    torch.save(checkpoint, path)  
    

epoch n° 1480 

eval_opportunity_cost_spat, Regression Loss: 0.0250, Regression Loss 1h: 312.5298, Segmentation Loss:0.0832
Train Confusion Matrix:
[[3147964   55100]
 [  70878  566058]]
Accuracy: 0.9672, CSI: 0.8180, Sensitivity: 0.8887, Specificity: 0.9828, False Alarm Ratio: 0.0887


eval_added_value_full_spat, Regression Loss: 0.0225, Regression Loss 1h: 313.7808, Segmentation Loss:0.0771
Train Confusion Matrix:
[[3149613   53451]
 [  64781  572155]]
Accuracy: 0.9692, CSI: 0.8287, Sensitivity: 0.8983, Specificity: 0.9833, False Alarm Ratio: 0.0854


train, Regression Loss: 0.0230, Regression Loss 1h: 324.2335, Segmentation Loss:0.0779
Train Confusion Matrix:
[[1960818   31507]
 [  43222  356773]]
Accuracy: 0.9688, CSI: 0.8268, Sensitivity: 0.8919, Specificity: 0.9842, False Alarm Ratio: 0.0811


epoch duration : 79.96184134483337
saving step
epoch n° 1481 

eval_opportunity_cost_spat, Regression Loss: 0.0238, Regression Loss 1h: 301.9496, Segmentation Loss:0.0894
Train Confusion Ma